# 🎯 모델 선택 및 합성 데이터에 대해 이야기해 봅시다

## 🛒 Zava 시나리오

Cora는 Zava 카탈로그에서 주택 개량 제품을 효과적으로 추천해야 합니다. 하지만 Cora에 어떤 AI 모델을 적용해야 할까요? 고급 추론 기능을 갖춘 GPT-4.1을 사용할까요, 아니면 비용 효율성을 고려한 GPT-4o-mini를 사용할까요?

**과제**: Cora를 실제 서비스에 배포하기 전에, 품질, 속도, 비용 측면에서 최적의 균형을 이루는 다양한 모델을 평가해야 합니다. 또한, 각 모델이 고객 문의에 대해 어떻게 작동하는지 비교할 수 있는 현실적인 테스트 데이터도 필요합니다.

## 학습 목표

이 섹션에서는 다음 내용을 이해하게 됩니다.

1. 사용 사례에 맞는 **다양한 AI 모델 선택 방법**
2. **합성 데이터셋이란 무엇이며 테스트에 유용한가**
3. **평가를 위한 질의-응답 쌍 생성 방법**
4. **RAG(Retrieval-Augmented Generation)란 무엇이며 응답 품질을 향상시키는가**
5. **모델 성능 비교를 위한 주요 평가 지표**

## 왜 이것이 중요한가?

적절한 모델을 선택하는 것은 다음과 같은 요소에 영향을 미치기 때문에 매우 중요합니다.
- **응답 품질** - Cora의 추천이 얼마나 정확하고 유용한가
- **비용** - 모델마다 가격 구조가 다릅니다.
- **지연 시간** - Cora가 고객에게 응답하는 속도
- **확장성** - 솔루션이 실제 운영 환경에서 트래픽을 처리할 수 있는지 여부

합성 데이터와 평가를 활용하여 정보에 입각한 모델 선택을 하는 방법을 살펴보겠습니다.

## 모델 선택이 중요한 이유

모든 언어 모델이 동일한 것은 아닙니다. 모델마다 강점이 다릅니다:

| Model | Best For | Trade-offs |
|-------|----------|------------|
| **GPT-4o** | 복잡한 추론, 멀티모달 작업 | 높은 비용, 느림 |
| **GPT-4o-mini** | 대량 처리와 속도가 중요한 애플리케이션 | 덜 세밀하지만 빠름 |
| **GPT-4.1** | 범용 목적, 우수한 성능 | 비용/품질 균형 |
| **Fine-tuned models** | 도메인 특화 작업, 일관된 스타일 | 학습 데이터 필요 |

**모델 선택의 주요 요인:**

### 1. 작업 복잡성
- **간단한 질문과 답변**: GPT-4o-mini가 탁월한 성능을 보입니다.
- **다단계 추론**: GPT-4o가 더 나은 결과를 제공합니다.
- **전문성**: 미세 조정을 고려해 보세요.

### 2. 비용 제약
모델은 토큰(입력 + 출력)당 비용을 청구합니다.
- GPT-4o-mini: 토큰당 비용이 낮고 처리량이 많습니다.
- GPT-4o: 토큰당 비용이 높지만 품질이 우수합니다.

**예시 계산:**
```
10,000 고객 질의/일
평균 500 토큰/대화

Model A (작은 모델): 토큰당 비용 낮음
Model B (큰 모델): 토큰당 비용 높음

대규모에서는 올바른 모델 선택이 상당한 비용 절감으로 이어질 수 있습니다.

Note: See Azure OpenAI Pricing for current rates:
https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/
```

### 3. 지연 시간 요구 사항
- **실시간 채팅**: GPT-4o-mini (빠른 응답)
- **백그라운드 처리**: GPT-4o (속도보다 품질 우선)

### 4. 정확도 요구 사항
- **사실적 정확성이 매우 중요** (예: 의료, 법률): GPT-4o + RAG
- **일반적인 지원** (예: 추천): GPT-4o-mini로 충분


**과제:** 어떤 모델이 *당신의* 사용 사례에 최적인지 어떻게 알 수 있을까? **해결책:** 테스트 데이터로 체계적 평가 수행.


---

## 합성 데이터란 무엇인가?

**합성 데이터**는 실제 시나리오를 모방해 인위적으로 생성한 테스트 데이터입니다.

### 합성 데이터를 생성하는 이유

**문제점:** 실제 고객 대화가 있기 전에는 테스트 데이터가 없음

**해결책:**
1. ❌ 실제 데이터 대기 → 프로덕션 환경 배포 전까지 테스트 불가
2. ❌ 수동 테스트 케이스 작성 → 시간 소모, 테스트 범위 제한적
3. ✅ 합성 데이터 생성 → 빠르고 확장 가능하며, 예외 상황 커버 가능

### 합성 데이터의 장점

**1. 조기 테스트**
- 배포 전 모델 테스트
- 개발 단계에서 문제 발견
- 개선 사항을 신속하게 반복 적용

**2. 포괄적인 테스트 범위**
- 수백 개의 테스트 케이스를 신속하게 생성
- 사람이 놓칠 수 있는 예외 상황 커버 가능
- 다양한 고객 의도 및 표현 방식 테스트

**3. 개인정보 보호 및 안전성**
- 실제 고객 데이터 불필요
- 개발/테스트에 안전
- 규정 준수 용이

**4. 비용 효율적**
- 수동 테스트 생성보다 빠름
- 실제 데이터를 기다리는 것보다 저렴함
- 반복 작업 전반에 걸쳐 재사용 가능

### 예시: Zava 제품 관련 질문

**실제 고객 질문**(수집하는 데 몇 달이 걸릴 수 있음):
```
"외부 목재용 페인트는 어떤 게 있나요?"
"콘크리트용 드릴이 필요한데, 어떤 걸 추천하시나요?"
"친환경 페인트 옵션이 있나요?"
"라텍스 페인트와 유성 페인트의 차이점은 무엇인가요?"
```

**합성 생성 예시** (몇 분 안에 생성 가능):
```python
# Azure AI Simulator generates similar questions from product catalog
simulator.generate_queries(
    source="product_catalog",
    count=100,
    variety=["product_search", "recommendations", "comparisons"]
)
```

**결과:** 100개의 현실적인 고객 질문(다양한 의도 포함)을 몇 초만에 얻을 수 있습니다.

---

## Azure AI Evaluation Simulator

**Azure AI 평가 시뮬레이터**는 데이터 소스를 기반으로 가상 쿼리-응답 쌍을 생성하는 도구입니다.

### How It Works

```
Your Data Sources           Simulator              Synthetic Dataset
                                ↓
┌─────────────────┐      ┌──────────────┐      ┌──────────────────┐
│ Product Catalog │  →   │  AI Simulator │  →   │ Query-Response   │
│ Documentation   │      │               │      │ Pairs (JSONL)    │
│ FAQs            │      │  Uses LLM to  │      │                  │
│ Knowledge Base  │      │  generate     │      │ 100+ realistic   │
└─────────────────┘      │  realistic    │      │ test examples    │
                         │  questions     │      └──────────────────┘
                         └──────────────┘
```

### What Gets Generated

**Query-Response Pairs** in JSONL format:

```json
{
  "query": "What paint is best for exterior wood?",
  "response": "For exterior wood, we recommend our Premium Exterior Paint (SKU: PFIP000002)...",
  "context": "Product: Premium Exterior Paint, Category: Paint & Finishes",
  "intent": "product_recommendation"
}
```

Each pair includes:
- **Query**: A customer question
- **Response**: Expected answer (generated from your data)
- **Context**: Source information used
- **Intent**: Type of question (optional metadata)

### Key Components

**1. Data Source Connection**
Connect to your knowledge base:
```python
# Example: Azure AI Search
search_client = SearchClient(
    endpoint=AZURE_SEARCH_ENDPOINT,
    index_name="products",
    credential=AzureKeyCredential(AZURE_SEARCH_API_KEY)
)
```

**2. RAG Application Callback**
Define how to retrieve information:
```python
def query_product_info(query: str) -> str:
    # Search product catalog
    results = search_client.search(query, top=3)
    # Format results
    return formatted_results
```

**3. Simulation Configuration**
Generate queries:
```python
simulator = Simulator(model_config=model_config)

outputs = simulator.generate(
    target=query_product_info,  # Your retrieval function
    num_queries=100,            # How many to generate
    max_conversation_turns=1    # Single Q&A or multi-turn
)
```

### Output Format: JSONL

**JSONL** (JSON Lines) = One JSON object per line

```jsonl
{"query": "What drill bits do you have?", "response": "We offer..."}
{"query": "Best paint for kitchens?", "response": "For kitchens..."}
{"query": "Do you have eco-friendly products?", "response": "Yes, we have..."}
```

**Why JSONL?**
- 스트리밍 및 라인별 처리 용이
- 머신러닝/인공지능 도구의 표준 형식
- 대규모 데이터셋에 효율적
- 평가 라이브러리와 호환 가능

---

## RAG: 검색 증강 생성

**RAG**는 응답을 생성하기 전에 관련 정보를 검색해 AI 응답을 개선하는 기법입니다.

### RAG 없을 때의 문제

**시나리오:** 고객이 "SKU PFIP000002가 무엇인가요?"라고 물을 때

**RAG 없이(모델만):**
```
Model: "I don't have information about specific SKUs in my training data."
```

모델은 특정 제품 정보를 알지 못합니다.

### The Solution: RAG

**RAG이 있는 경우:**
```
1. 검색: 제품 데이터베이스에서 "PFIP000002" 검색
   → 찾음: "Premium Exterior Paint, $45.99, In Stock"
   
2. 보강: 검색한 정보를 프롬프트에 추가
   "다음 제품 정보를 바탕으로 답하세요: [Premium Exterior Paint...]"
   
3. 생성: 모델이 응답 생성
   → "SKU PFIP000002는 Premium Exterior Paint이며 가격은 $45.99이고 현재 재고가 있습니다."
```

**결과:** 실제 데이터에 근거한 정확한 응답

### RAG 작동 방식

```
Customer Question
      ↓
┌─────────────────┐
│  1. Retrieve    │  지식베이스 검색
│                 │  (Azure AI Search, Vector DB 등)
└────────┬────────┘
         ↓
   Retrieved Context
         ↓
┌─────────────────┐
│  2. Augment     │  질문 + 컨텍스트를 프롬프트에 결합
│                 │  "Based on: [context], answer: [question]"
└────────┬────────┘
         ↓
   Enhanced Prompt
         ↓
┌─────────────────┐
│  3. Generate    │  LLM이 컨텍스트를 사용해 응답 생성
│                 │  
└────────┬────────┘
         ↓
   Factual Response
```

### RAG vs. 파인튜닝 vs. 프롬프트

| Approach | Use When | Pros | Cons |
|----------|----------|------|------|
| **RAG** | 데이터가 자주 변경될 때 | 항상 최신 정보 기반, 사실성 높음 | 검색 인프라 필요 |
| **Fine-Tuning** | 스타일/톤 일관성이 필요할 때 | 효율적, 검색 불필요 | 정적 지식, 재학습 필요 |
| **Prompting** | 단순 작업, 정적 정보 | 빠르고 인프라 불필요 | 컨텍스트 길이 제한 |

**Cora의 경우:** 제품 정보(자주 변경)에 대해서는 RAG를 사용하고, 톤(브랜드 스타일) 일관성을 위해 파인튜닝을 결합하세요.

### 모델 평가에 대한 이점

시뮬레이터로 합성 데이터 생성 시:
- **RAG는 응답을 근거 기반으로 만듭니다** - 실제 제품 데이터 기반 응답
- **현실적인 테스트 케이스** - 질문이 실제 카탈로그를 반영
- **측정 가능한 정확성** - 소스 데이터와 응답을 대조 가능

이 때문에 시뮬레이터는 RAG 콜백을 사용해 현실적이고 검증 가능한 테스트 데이터를 생성합니다.

---

## 핵심 개념: 질의, 응답, 그리고 쌍

### 질의

**질의**는 사용자의 질문이나 요청입니다.

**예시:**
- "What paint do you have for exterior wood?"
- "I need a drill, what do you recommend?"
- "Is SKU PFIP000002 in stock?"

**질의의 특징:**
- **의도(Intent)**: 사용자가 원하는 것(검색, 추천, 사실 확인 등)
- **복잡도**: 단순 vs 다부분 질문
- **특이성**: 광범위 vs 특정

### 응답

**응답**은 AI 모델이 생성한 답변입니다.

**예시:**
```
Query: "What paint do you have for exterior wood?"

Response: "For exterior wood projects, I recommend our Premium Exterior Paint 
(SKU: PFIP000002). It's weather-resistant, durable, and available in multiple 
colors. Currently priced at $45.99 with 75 units in stock."
```

**품질 요인:**
- **정확성**: 사실 여부
- **완전성**: 질문에 대한 전체 답변 여부
- **관련성**: 주제를 벗어나지 않는지
- **도움성**: 유용한 정보 제공 여부
- **톤**: 브랜드 보이스와 일치 여부 (예: 정중, 전문적)

### 질의-응답 쌍

**Pair**은 질의와 예상 응답을 결합한 테스트 케이스입니다.

**Format:**
```json
{
  "query": "What paint is best for exterior wood?",
  "response": "For exterior wood, I recommend Premium Exterior Paint (SKU: PFIP000002)...",
  "ground_truth": "Premium Exterior Paint",
  "context": "Products: PFIP000002, PFIP000003, PFIP000005"
}
```

**Pair의 장점:**
- **비교 기준 제공**: 예상 vs 실제 응답
- **재현 가능한 테스트**: 모델 버전 간 동일한 질의 사용 가능
- **품질 지표 측정**: 응답이 기대치와 얼마나 일치하는지 측정 가능

### 평가 워크플로우

```
1. Generate Pairs (Simulator)
   → 100 query-response pairs from product catalog

2. Test Model A (GPT-4o-mini)
   → Run 100 queries through model
   → Collect 100 responses

3. Test Model B (GPT-4o)
   → Run same 100 queries through different model
   → Collect 100 responses

4. Compare Results
   → Which model's responses better match expected responses?
   → Which is more accurate, helpful, relevant?

5. Select Winner
   → Choose model based on metrics
```

---

## 모델 선택을 위한 평가 지표

어떤 모델이 더 나은지 측정하려면 다음 지표를 사용하세요:

### 1. 근거성(Groundedness)

**측정 내용:** 응답이 검색된 컨텍스트에 기반했는가(환각이 아닌가)?,

**예시:**
```
Context: "Premium Exterior Paint costs $45.99"

Good (grounded): "Premium Exterior Paint is priced at $45.99"
Bad (not grounded): "Premium Exterior Paint costs around $40"
```

**점수:** 0에서 5까지 (5 = 완전히 컨텍스트에 기반)

### 2. 관련성(Relevance)

**측정 내용:** 응답이 질의에 부합하는가?,

**예시:**
```
Query: "What paint is best for exterior wood?"

Good (relevant): "For exterior wood, Premium Exterior Paint is ideal..."
Bad (irrelevant): "We have many paint options in different colors..."
```

**점수:** 0에서 5까지 (5 = 완벽히 관련됨)

### 3. 일관성(Coherence)

**측정 내용:** 응답이 잘 구성되고 논리적인가?,

**예시:**
```
Good (coherent): "Premium Exterior Paint is durable and weather-resistant, 
making it perfect for outdoor wood surfaces."

Bad (incoherent): "Paint wood exterior durable Premium weather yes outdoor."
```

**점수:** 0에서 5까지 (5 = 완벽히 일관됨)

### 4. 유창성(Fluency)

**측정 내용:** 언어가 자연스럽고 문법적으로 올바른가?,

**예시:**
```
Good (fluent): "I recommend Premium Exterior Paint for your project."
Bad (not fluent): "I recommend you Premium Exterior Paint is for project."
```

**점수:** 0에서 5까지 (5 = 완벽한 문법과 자연스러움)

### 5. 유사성(Similarity to Expected Response)

**측정 내용:** 실제 응답이 예상 응답에 얼마나 가까운가?,

**측정 방법:**
- 코사인 유사도(임베딩 벡터)
- BLEU 점수(텍스트 중복도)
- 의미 유사성(의미적 일치)

**예시:**
```
Expected: "Premium Exterior Paint costs $45.99 and is in stock."
Actual:   "Our Premium Exterior Paint is priced at $45.99 with availability."

Similarity: 0.92 (very similar)
```

### 지표 결합하기

**모델 평가 스코어카드:**

| Metric | GPT-4o-mini | GPT-4o | Winner |
|--------|-------------|--------|--------|
| Groundedness | 4.2 | 4.8 | GPT-4o |
| Relevance | 4.5 | 4.7 | GPT-4o |
| Coherence | 4.3 | 4.6 | GPT-4o |
| Fluency | 4.6 | 4.7 | GPT-4o |
| Similarity | 0.85 | 0.91 | GPT-4o |
| **Avg Latency** | **800ms** | **1200ms** | **GPT-4o-mini** |
| **Cost/1K queries** | **Lower** | **Higher** | **GPT-4o-mini** |

**참고:** 현재 가격은 Azure OpenAI Pricing 확인

**Decision:** 
- 품질이 중요하면 → GPT-4o 선택

- 속도/비용이 중요하면 → GPT-4o-mini- 하이브리드: 단순 질의에 GPT-4o-mini, 복잡한 경우 GPT-4o 사용

---

## 모델 선택 프로세스

### 1단계: 요구사항 정의

**자기 점검 항목:**
- 내 작업은 무엇인가? (Q&A, 추천, 복잡한 추론 등)
- 허용 가능한 품질 수준은?
- 예산은?
- 허용 가능한 지연 시간은?
- 하루 질의 수는 얼마나 되는가?

**Cora 예시:**
- 작업: 상품 Q&A, 재고 확인
- 품질: 높은 정확성 요구(사실성 중요)
- 예산: 중간(하루 수천 건의 질의 예상)
- 지연: < 2초 선호
- 볼륨: 약 5,000 질의/일

### 2단계: 테스트 데이터 생성

**Azure AI Simulator 사용:**
```python
# 제품 카탈로그에서 100개의 질의-응답 쌍 생성
dataset = simulator.generate(
    target=rag_callback,
    num_queries=100
)

# JSONL로 저장
with open("test_dataset.jsonl", "w") as f:
    for item in dataset:
        f.write(json.dumps(item) + "\n")
```

**결과:** 100개의 현실적인 고객 질문과 예상 답변

### 3단계: 후보 모델 테스트

**각 모델을 테스트 데이터로 실행:**

```python
# GPT-4o-mini 테스트
results_mini = []
for item in test_dataset:
    response = model_mini.query(item["query"])
    results_mini.append({
        "query": item["query"],
        "response": response,
        "expected": item["response"]
    })

# GPT-4o 테스트
results_4o = []
for item in test_dataset:
    response = model_4o.query(item["query"])
    results_4o.append({
        "query": item["query"],
        "response": response,
        "expected": item["response"]
    })
```

### 4단계: 결과 평가

**평가 지표 사용:**

```python
from azure.ai.evaluation import evaluate

# GPT-4o-mini 평가
eval_mini = evaluate(
    data=results_mini,
    evaluators={
        "groundedness": groundedness_evaluator,
        "relevance": relevance_evaluator,
        "coherence": coherence_evaluator
    }
)

# GPT-4o 평가
eval_4o = evaluate(
    data=results_4o,
    evaluators={...}
)
```

### 5단계: 비교 및 결정

**비교 생성:**

```python
comparison = {
    "GPT-4o-mini": {
        "quality": eval_mini.average_scores,
        "cost": 0.50,
        "latency": 800
    },
    "GPT-4o": {
        "quality": eval_4o.average_scores,
        "cost": 2.50,
        "latency": 1200
    }
}
```

**결정 프레임워크:**

```
If quality_difference < 0.3 AND cost_difference > 2x:
    → Choose cheaper model (GPT-4o-mini)
    
If quality_difference > 0.5:
    → Choose better model (GPT-4o)
    
Else:
    → Consider hybrid approach
```


### 6단계: 반복

- 더 많은 데이터로 테스트 확장
- 다른 프롬프트 시도
- 파인튜닝 고려
- 주기적으로 재평가

---

## 모범 사례

### 1. 다양한 테스트 케이스 생성

```python
# Good - covers different intents
queries = [
    "What paint is best for exterior wood?",      # Recommendation
    "Is SKU PFIP000002 in stock?",                # Fact check
    "Compare latex vs oil-based paint",           # Comparison
    "I need eco-friendly options",                # Filtered search
    "What's the price of Premium Exterior Paint?" # Specific fact
]

# Less effective - repetitive
queries = [
    "What paint do you have?",
    "Do you sell paint?",
    "Tell me about paint",
    # All similar intent
]
```

### 2. 현실적인 표현 사용

고객이 실제로 말하는 방식과 유사한 질의를 생성하세요:

```python
# Realistic
"I'm painting my deck, what should I use?"
"Need something for outdoor wood"
"Best paint for weather resistance?"

# Too formal (less realistic)
"Please provide recommendations for exterior wood coating solutions"
```

### 3. 엣지 케이스 포함

```python
# Test edge cases
"Do you have paint?" # Vague
"I need PFIP000002 but in blue" # Specific constraint
"What's the cheapest paint?" # Price-focused
"" # Empty query
"ajshdkajhsd" # Gibberish
```

### 4. 데이터셋 크기 균형

- **너무 작음** (< 20 질의): 대표성 부족
- **적절** (50-100 질의): 균형 잡힌 커버리지
- **대규모** (500+ 질의): 포괄적이지만 느리고 비용 증가

**시작은 50-100로 하고 필요시 확장하세요**

### 5. 데이터셋 버전 관리

```python
# Save with version numbers
"test_dataset_v1.jsonl"  # Initial
"test_dataset_v2.jsonl"  # Added edge cases
"test_dataset_v3.jsonl"  # Added multi-turn conversations
```

이렇게 하면 시간에 따른 모델 성능 비교가 가능합니다.

---

## 용어 빠른 참조

| Term | Simple Definition |
|------|-------------------|
| **Synthetic Dataset** | 실제 시나리오를 모방한 인위적 테스트 데이터 |
| **Query** | 사용자 질문 또는 요청 |
| **Response** | AI 모델이 생성한 답변 |
| **Query-Response Pair** | 질의와 예상 응답을 결합한 테스트 케이스 |
| **RAG** | Retrieval-Augmented Generation - 응답 생성 전 관련 정보 검색 |
| **JSONL** | JSON Lines 형식 - 한 줄에 하나의 JSON 객체 |
| **Groundedness** | 응답이 제공된 컨텍스트에 기반했는지 측정하는 지표 |
| **Relevance** | 응답이 질의에 부합하는지 측정하는 지표 |
| **Coherence** | 응답의 구조적 일관성을 측정하는 지표 |
| **Fluency** | 응답의 자연스러움과 문법 정확성 측정 |
| **Similarity** | 실제 응답과 예상 응답의 비교 지표 |
| **Simulator** | 합성 테스트 데이터를 생성하는 도구 |
| **Latency** | 응답 생성에 걸리는 시간 |
| **Token** | 청구 단위의 텍스트 (~4문자) |

---

## 다음 단계

이제 모델 선택 개념을 이해했으니, 테스트 데이터 생성 및 평가로 넘어갈 준비가 되었습니다!

### 이 섹션의 실습 노트북

- **`21-simulate-dataset.ipynb`** - Azure AI Simulator로 합성 테스트 데이터 생성
  - Azure AI Search에 연결
  - RAG 애플리케이션 콜백 생성
  - 질의-응답 쌍 생성
  - JSONL 형식으로 데이터셋 저장

- **`22-evaluate-models.ipynb`** - 생성한 데이터셋으로 모델 비교
  - 여러 모델에 질의 실행
  - 평가 지표 계산
  - 품질, 비용, 지연 시간 비교
  - 데이터 기반 모델 선택

### 권장 학습 경로

1. **여기에서 시작** → 개념 이해(이 노트북)
2. **다음** → 테스트 데이터 생성(`21-simulate-dataset.ipynb`)
3. **그 다음** → 모델 평가(`22-evaluate-models.ipynb`)
4. **그 이후** → 커스터마이제이션 실습(파인튜닝, 증류 등)
5. **마지막으로** → 선택한 모델 배포 및 모니터링

---

## 추가 자료

더 깊이 있는 학습을 위해:

- **[Azure AI Evaluation SDK](https://learn.microsoft.com/azure/ai-studio/how-to/develop/evaluate-sdk)** - 공식 평가 가이드
- **[Azure AI Simulator](https://learn.microsoft.com/azure/ai-studio/how-to/develop/simulator-interaction-data)** - 합성 데이터 생성 가이드
- **[RAG 개요](https://learn.microsoft.com/azure/ai-studio/concepts/retrieval-augmented-generation)** - 검색 증강 생성 개념
- **[모델 선택 가이드](https://learn.microsoft.com/azure/ai-services/openai/concepts/models)** - 모델 선택 관련 자료
- **[평가 지표](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-metrics-built-in)** - 품질 지표 설명

---

테스트 데이터 생성을 시작할 준비가 되셨나요? `21-simulate-dataset.ipynb`를 열어 시작하세요! 🚀